# 🚀 LUMPS - Worker Distribuido para Google Colab

Este notebook permite ejecutar un worker distribuido para generar el dataset de la Fase 0 de LUMPS.

## 📋 Instrucciones

1. **Ejecuta las celdas en orden**
2. **Configura tu ruta de Google Drive** en la celda de configuración
3. **El worker trabajará automáticamente** hasta que se desconecte Colab
4. **Puedes pausar/reanudar** usando los botones de control

## ⚠️ Importante
- Cada usuario debe hacer **"Archivo > Guardar una copia en Drive"** antes de ejecutar
- El worker trabajará en **loop continuo** hasta desconexión
- Los resultados se guardan automáticamente en Google Drive compartido


## 🔧 Setup Inicial


In [ ]:
# Montar Google Drive
from google.colab import drive
import os

print("🔗 Montando Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive montado exitosamente")


In [ ]:
# Clonar repositorio desde GitHub
import subprocess
import sys
from pathlib import Path

print("📥 Clonando repositorio LUMPS...")

# Cambiar al directorio de trabajo
%cd /content

# Clonar repositorio (reemplaza con tu URL de GitHub)
repo_url = "https://github.com/juanignaciorey/LUMPS.git"  # ⚠️ CAMBIA ESTA URL
!git clone $repo_url

# Cambiar al directorio del proyecto
%cd LUMPS

print("✅ Repositorio clonado exitosamente")
print(f"📁 Directorio actual: {os.getcwd()}")


In [ ]:
# Instalar dependencias
print("📦 Instalando dependencias...")

# Instalar requirements
!pip install -r requirements.txt

# Instalar dependencias adicionales para Colab
!pip install tqdm

print("✅ Dependencias instaladas exitosamente")


## ⚙️ Configuración


In [ ]:
# Configurar paths y variables
import os
import socket
import time
from pathlib import Path

# ⚠️ CONFIGURAR ESTA RUTA - debe ser la carpeta compartida en Drive
DRIVE_BASE_PATH = "/content/drive/MyDrive/LUMPS_Distributed"

# Generar ID único para este worker
WORKER_ID = f"colab_{socket.gethostname()}_{os.getpid()}_{int(time.time())}"

# Configurar variables de entorno
os.environ['LUMPS_DRIVE_PATH'] = DRIVE_BASE_PATH
os.environ['WORKER_ID'] = WORKER_ID

print(f"🔧 Configuración:")
print(f"   Worker ID: {WORKER_ID}")
print(f"   Drive Path: {DRIVE_BASE_PATH}")
print(f"   Hostname: {socket.gethostname()}")
print(f"   PID: {os.getpid()}")

# Verificar que la ruta existe
if not Path(DRIVE_BASE_PATH).exists():
    print(f"❌ Error: La ruta {DRIVE_BASE_PATH} no existe")
    print("💡 Asegúrate de que la carpeta compartida esté en tu Drive")
else:
    print(f"✅ Ruta verificada: {DRIVE_BASE_PATH}")


## 🖥️ Configuración de GPU


In [ ]:
# Verificar GPU disponible
import torch

print("🖥️ Verificando GPU...")
print(f"   CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("✅ GPU configurada y lista")
else:
    print("⚠️ No hay GPU disponible, usando CPU")

print(f"   PyTorch versión: {torch.__version__}")


## 🚀 Ejecutar Worker


In [ ]:
# Ejecutar worker distribuido
print("🚀 Iniciando worker distribuido...")
print("⚠️ El worker trabajará en loop continuo hasta desconexión")
print("🛑 Presiona el botón de interrumpir para detener")
print("\n" + "="*80)

# Ejecutar worker con TQDM para progreso visual
!python generate_dataset_distributed.py \
    --drive-path $LUMPS_DRIVE_PATH \
    --worker-id $WORKER_ID \
    --verbose
